In [1]:
import pandas as pd
import numpy as np
import logging


class Datapoint():
    def __init__(self, km, price):
        self.km = km
        self.price = price


    def __str__(self):
        return (f"km: {self.km}, price {self.price}")


    def __repr__(self):
        return (f"km: {self.km}, price {self.price}")

class Dataset():
    def __init__(self, path):
        self.data = None
        self.i = -1
        self.read_csv(path)


    def read_csv(self, path):
        self.data = pd.read_csv(path).to_numpy()
        try:
            self.p = self.data.shape[1] - 1
            self.m = self.data.shape[0]
        except:
            logging.error(f"Input needs to have at least two dimensions. Input dimension was {self.data.shape}")
            raise ValueError

        self.x = self.data[:, [x for x in range(self.p)]]
        self.x = np.concatenate((np.ones([self.m, 1], dtype = self.x.dtype), self.x), axis = 1)
        self.y = self.data[:, self.p]
        self.x_sum = np.sum(self.x)
        self.y_sum = np.sum(self.y)
        self.x = self.x / self.x_sum
        self.y = self.y / self.y_sum



    def __getitem__(self, i):
        return (self.x[i], self.y[i])
    

    def __len__(self):
        return (self.data.shape[0])

    
    def __iter__(self):
        self.i = -1
        return (self)

    def __next__(self):
        self.i += 1
        if (self.i < len(self)):
            return self[self.i]
        else:
            self.i = -1
            raise StopIteration

In [24]:
float_formatter = "{:.2E}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

def normalize(a):
    col_sums = a.sum(axis=0)
    new_matrix = a / col_sums[np.newaxis, :]
    return new_matrix

class Shaman():
    def __init__(self, dataset):
        self.p = dataset.p
        self.dataset = dataset
        self.thetas = np.zeros([self.p + 1], dtype = float)
        self.old_thetas = self.thetas
        self.lr = 1.0
        self.lr_decay = 1.0 / 2
        self.mininal_improvement = 0.1
        self.newcost = 0.0
        self.oldcost = 0.0
        self.c = 1.0 / 2
        self.tau = 1.0 / 2
        self.lr_increase = 1.5


    def predict(self, data, thetas = None):
        if thetas is None:
            thetas = self.thetas
        return (np.dot(data, thetas))


    def error(self, thetas = None):
        if thetas is None:
            thetas = self.thetas
        predictions = self.predict(self.dataset.x, thetas)
        error = predictions - self.dataset.y
        return (error)


    def mean_squared_error(self, thetas = None):
        if thetas is None:
            thetas = self.thetas   
        squared_error = np.square(self.error(thetas))
        return (np.mean(squared_error) / 2)


    def compute_gradients(self):
        error = self.error()
        gradients = np.dot(error, normalize(self.dataset.x))
        gradients = gradients / len(self.dataset.y)
        return (gradients)


    def ajimo_goldstein_condition(self, l2_grad_squared, gradients, lr):
        thetas = self.thetas - lr * gradients
        cost = self.mean_squared_error(thetas)
        objective = self.newcost - (self.c * lr * l2_grad_squared)
        return (cost <= objective)


    def ajimo(self, gradients):
        l2_grad_squared = np.square(gradients).sum()
        lr = self.lr * self.lr_increase
        while (not self.ajimo_goldstein_condition(l2_grad_squared, gradients, lr)):
            lr = lr * self.lr_decay
        self.lr = lr

    def update_thetas(self):
        self.old_thetas = self.thetas
        gradients = self.compute_gradients()
        self.ajimo(gradients)
        self.thetas = self.thetas - (self.lr * gradients)

    
    def update_costs(self):
        tmpold = self.oldcost
        self.oldcost = self.newcost
        self.newcost = self.mean_squared_error()
        return tmpold


    def undo_update_costs(self, tmpold):
        self.newcost =self.oldcost
        self.oldcost = tmpold


    def training_loop(self):
        keep_learning = True
        self.newcost = self.mean_squared_error()
        while (keep_learning):
            tmpold = self.oldcost
            self.update_thetas()
            tmpold = self.update_costs()
            if (self.newcost > self.oldcost):
                self.lr = self.lr * self.lr_decay
                self.thetas = self.old_thetas
                self.undo_update_costs(tmpold)
            keep_learning = not self.should_i_stop()
            # print(self)


    def middle_error(self):
        middle_thetas = (self.thetas + self.old_thetas) / 2
        return self.mean_squared_error(middle_thetas)

    
    def should_i_stop(self):
        if abs(self.oldcost - self.newcost) > self.mininal_improvement:
            return False
        if abs(self.middle_error() - self.newcost) > self.mininal_improvement:
            return False
        return True


    def __str__(self):
        return (f"Cost: {self.newcost:.2e}, Thetas: {self.thetas}, LR {self.lr:4.2E}")


In [25]:
from datetime import datetime
now = datetime.now()
d = Dataset("data.csv")
shaman = Shaman(d)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
shaman.training_loop()
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
# shaman.predict(d.x[0])
# shaman.predict(d.x)
# print(shaman)
# shaman.update_thetas(d)
# print(shaman)
# shaman.compute_gradients(d)

Current Time = 21:04:13
Current Time = 21:05:04


In [26]:
print(shaman)
for x, y in shaman.dataset:
    print(f"{shaman.predict(x):.0f}\t {y}")

Cost: 2.54e+05, Thetas: [7.95E+03 -1.71E-02], LR 6.35E-04
3839	 3650
5556	 3800
5372	 4400
4772	 4450
4935	 5250
5984	 5350
5093	 5800
6426	 5990
5475	 5999
6511	 6200
6545	 6390
6870	 6390
6683	 6600
6280	 6800
6803	 6800
6648	 6900
7124	 6900
6357	 6990
6906	 7490
6825	 7555
7025	 7990
6777	 7990
7558	 7990
6892	 8290


In [195]:
normalize(d.x)
# d.x / d.x.sum(axis=0)[np.newaxis, :]

array([[0.04166667, 0.098945  ],
       [0.04166667, 0.05763546],
       [0.04166667, 0.06204676],
       [0.04166667, 0.07648861],
       [0.04166667, 0.07255967],
       [0.04166667, 0.04732869],
       [0.04166667, 0.06876677],
       [0.04166667, 0.0366921 ],
       [0.04166667, 0.05957313],
       [0.04166667, 0.03463075],
       [0.04166667, 0.03381816],
       [0.04166667, 0.0259978 ],
       [0.04166667, 0.03050804],
       [0.04166667, 0.04019641],
       [0.04166667, 0.02762215],
       [0.04166667, 0.03134289],
       [0.04166667, 0.01988588],
       [0.04166667, 0.03834119],
       [0.04166667, 0.02512749],
       [0.04166667, 0.02707547],
       [0.04166667, 0.02226262],
       [0.04166667, 0.02824055],
       [0.04166667, 0.00944059],
       [0.04166667, 0.0254738 ]])

In [138]:
e = np.array([1,2,3,4])
inputs = np.array([[1,2,3], [2,2,2], [3,3,3], [4,4,4]])

In [139]:
print(e)
print("\n")
print(inputs)

[1 2 3 4]


[[1 2 3]
 [2 2 2]
 [3 3 3]
 [4 4 4]]


In [140]:
np.dot(e, inputs)

array([30, 31, 32])

In [141]:
print(d.x)

[[     1 240000]
 [     1 139800]
 [     1 150500]
 [     1 185530]
 [     1 176000]
 [     1 114800]
 [     1 166800]
 [     1  89000]
 [     1 144500]
 [     1  84000]
 [     1  82029]
 [     1  63060]
 [     1  74000]
 [     1  97500]
 [     1  67000]
 [     1  76025]
 [     1  48235]
 [     1  93000]
 [     1  60949]
 [     1  65674]
 [     1  54000]
 [     1  68500]
 [     1  22899]
 [     1  61789]]
